# 📡 06 - Model Inference to Alerts

This notebook uses a trained anomaly detection model to infer risk scores from sensor feature data, 
filters high-risk records (anomaly = 1), and writes alerts into a timestamped Delta table.

**Steps:**
- Read predictions from the latest model
- Filter only anomalous records
- Write them to a time-versioned table for downstream alerting or monitoring


In [0]:
from pyspark.sql.functions import col, to_timestamp, to_date, lit
from datetime import datetime

timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")

predictions = spark.read.table("arao.aerodemo.prediction_results")

predictions_casted = (
    predictions
    .withColumn("timestamp", col("timestamp").cast("timestamp"))  # ensure it's cast
    .withColumn("alert_day", to_date("timestamp"))                # <-- this is key
    .withColumn("aircraft_id", col("aircraft_id").cast("string"))
    .withColumn("engine_rpm", col("engine_rpm").cast("int"))
    .withColumn("capacity", col("capacity").cast("int"))
    .withColumn("range_km", col("range_km").cast("int"))
    .withColumn("predicted_anomaly", col("predicted_anomaly").cast("long"))
    .withColumn("batch_id", lit(timestamp_str))
)

# Drop and recreate
spark.sql("DROP TABLE IF EXISTS arao.aerodemo.engine_alerts")
predictions_casted.limit(0).write.format("delta").mode("overwrite").saveAsTable("arao.aerodemo.engine_alerts")

# Filter alerts only
alerts = predictions_casted.filter(col("predicted_anomaly") == 1)

alerts.write.format("delta").mode("append").saveAsTable("arao.aerodemo.engine_alerts")

In [0]:
%sql
SELECT DISTINCT DATE(alert_day) FROM arao.aerodemo.engine_alerts

In [0]:
spark.table("arao.aerodemo.engine_alerts").printSchema()

### 📌 Notes:
- `alert_generated_at`: records the timestamp of alert generation
- `batch_id`: uniquely identifies the batch for traceability or rollback
- This table can be used for integration with dashboards, alerting systems, or compliance reporting
